# 9. Proper Motions - IN DEVELOPMENT -

This notebook takes a look at proper motions (discrete data). In its current state, it will **read and display discrete data** of an example galaxy "F2" (proper motions as two-dimensional histogram data), create a model for that galaxy "F2" based on losvd data only, and **calculate and display the DYNAMITE model's proper motion data**.

*Note that currently there is no weight solving involving the discrete data yet.*

Please note that this notebook is still in development, as is the DYNAMITE functionality pertaining to discrete data. For more information on the options and functionality of the methods described here, please refer to the API docs.

## 9.1. Prerequisites

Import the required modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cmasher as cmr
import dynamite as dyn
from plotbin import display_pixels

## 9.2. Input Data

Read the DYNAMITE input. The configuration file contains GaussHermite losvd kinematics and proper motion data.
If there are models present in the output folder, the all_models table will be read as well.

In [ ]:
# Get the DYNAMITE models
fname = 'F2_config11_with_pm.yaml'
# fname = 'F2_config11.yaml'
c = dyn.config_reader.Configuration(fname,
                                    reset_logging=True,
                                    user_logfile='test_nnls',
                                    reset_existing_output=False)

Let's have a look at the kinematics, especially the proper motions data...

In [ ]:
stars = c.system.get_unique_triaxial_visible_component()
print(f'There are {len(stars.kinematic_data)} kinematic datasets: ' \
      f'{[k.name for k in stars.kinematic_data]}')
pm = [k for k in stars.kinematic_data
      if isinstance(k, dyn.kinematics.ProperMotions)][0]  # get ProperMotions
pm_data = pm.get_data()
print('The proper motions data (pm_data=pm.get_data()) read from input are:')
npz_data = ['PM_2dhist', 'PM_2dhist_sigma', 'binID_dynamite', 'nstarbin', \
            'vxrange', 'vyrange', 'xbin', 'ybin']
for k in npz_data:
    if pm_data[k].ndim > 0:
        print(f'\tpm_data["{k}"].shape = {pm_data[k].shape}')
    else:
        print(f'\tpm_data["{k}"] = {pm_data[k]}')
print('Data calculated for proper motions normalisation:')
print(f'\tpm_data["hist_scale"] = {pm_data["hist_scale"]}')
print('UNUSED data in current npz input file:')
for k in [k for k in pm_data.keys() if k not in npz_data + ['hist_scale']]:
    if pm_data[k].ndim > 0:
        print(f'\tpm_data["{k}"].shape = {pm_data[k].shape}')
    else:
        print(f'\tpm_data["{k}"] = {pm_data[k]}')
print('Other data in ProperMotions attributes:')
print(f'\t{pm.hist_bins = }')
print(f'\t{pm.hist_width = }')
print(f'\t{pm.hist_center = }')

There should be no empty apertures / spatial bins:

In [ ]:
min_hist = np.min(np.sum(pm_data['PM_2dhist'], axis=(1, 2)))
print(f'There are {"no " if min_hist > 0 else ""}empty spatial bins.')

In the models, each orbit will result in a 2d velocity histogram for each aperture, stored in a Histogram2D object. For visualizing the input data, it can be converted into a Histogram2D object (with a trivial orbit dimension of 1):

In [ ]:
h2d = pm.as_histogram2d()
print(f'vx edges: {h2d.xedg[0].shape = }\t {np.min(h2d.xedg[0]) = }, {np.max(h2d.xedg[0]) = }')
print(f'vy edges: {h2d.xedg[1].shape = }\t {np.min(h2d.xedg[1]) = }, {np.max(h2d.xedg[1]) = }')
print(f'vx bin centers: {h2d.x[0].shape = }\t {np.min(h2d.x[0]) = }, {np.max(h2d.x[0]) = }')
print(f'vy bin centers: {h2d.x[1].shape = }\t {np.min(h2d.x[1]) = }, {np.max(h2d.x[1]) = }')
print(f'vx bin widths: {h2d.dx[0].shape = }\t {np.min(h2d.dx[0]) = }, {np.max(h2d.dx[0]) = }')
print(f'vy bin widths: {h2d.dx[1].shape = }\t {np.min(h2d.dx[1]) = }, {np.max(h2d.dx[1]) = }')
print(f'velocity histogram values: {h2d.y.shape = }')

Let's plot the proper motions...

In [ ]:
p = dyn.plotter.Plotter(c)

In [ ]:
# First, a few individual spatial bins...
# Notes: orb_idx=0 because the input data is not given by orbit
#        empty_bins=True will indicate empty velocity bins with "x"
for sp_bin_idx in(11, 48, 75):
    print(f'{sp_bin_idx=}')
    _ = p.hist2d_plot(h2d, orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)
# Now the global 2d histogram...
h2d_global = dyn.kinematics.Histogram2D(xedg=h2d.xedg,
                                        y=np.sum(h2d.y, axis=3)[:,:,:,np.newaxis])
_ = p.hist2d_plot(h2d_global, orb_idx=0, sp_bin_idx=0, show_1d=True, empty_bins=True)

In [ ]:
# Experimental, does NOT work yet: quiver plot -> problem in get_mean or plotting...?
v_mean = h2d.get_mean()
x, y = pm_data['xbin'], pm_data['ybin']
u, v = v_mean[0][0], v_mean[1][0]  # first index: vx/vy, second index: orb_idx
u_average = np.average(v_mean[0][0], weights=pm_data['nstarbin'])
v_average = np.average(v_mean[1][0], weights=pm_data['nstarbin'])
plt.quiver(x, y, u - u_average, v - v_average, angles='xy', color='g')
plt.axis('equal')

## 9.3. Inspect Orblib Proper Motions

In order to get the orblibs, at least one model needs to be run. In the present case, there are models in the output directory already, so nothing needs to be done:

In [ ]:
_ = dyn.model_iterator.ModelIterator(config=c)

Pick the best model and get its orblib and weights.
- If the model has been calculated alrady using only losvd but no proper motions, then the orblib will be recalculated, writing the 2d histogram files `orblib_pm_hist.dat.bz2` and `orblibbox_pm_hist.dat.bz2`, respectively.
- If the weights file exists from a previous run, it will be used to populate `model.weights`.

In [ ]:
# Get best model
best_model_idx = c.all_models.get_best_n_models_idx(n=1)[0]
best_model_idx = 13  # here, we deliberately override to illustrate velocity scaling below
print(best_model_idx)
model = c.all_models.get_model_from_row(best_model_idx)
orblib = model.get_orblib()
print(f'{orblib.parset = }')
_ = model.get_weights(orblib)
orb_weights = model.weights
print(f'Size of model orblib: {len(orb_weights)}')

For reading the 2d histograms of the orbits' proper motions, we need the index of the proper motions kinematics data.

In [ ]:
# Get the index of the proper motions kinemtics data
pm_idx = [i[0] for i in enumerate(stars.kinematic_data)
          if isinstance(i[1], dyn.kinematics.ProperMotions)][0]
print(f'Proper motions is kinematics set no. {pm_idx}.')


Now we can read the velocity histograms from disk and populate the `Histogram2D` object `h2d_orblib`, the orbit library's 2d histograms:

In [ ]:
orblib.read_vel_histograms()
h2d_orblib = orblib.vel_histograms[pm_idx]
print(f'Read a {type(h2d_orblib)} object.')
print(f'{h2d_orblib.y.shape = }')

Depending on the model's `ml` parameter, a velocity scaling factor might have been used in the model. Therefore, the velocity histograms may need to be rebinned, as it is the case in the displayed example.

Note that during normal operation, DYNAMITE handles this internally without the need for user intervention.

In [ ]:
# Get h2d_model, the weights-weighted sum of the orblib contributions
data = np.einsum('ijkl,i', h2d_orblib.y, orb_weights)  # weights-weighted sum
h2d_model = dyn.kinematics.Histogram2D(xedg=h2d_orblib.xedg, y=data[np.newaxis])
h2d_rebinned = dyn.kinematics.Histogram2D(xedg=h2d.xedg,
                                          y=pm.rebin_orblib_to_observations(h2d_model))
for sp_bin_idx in (29,):  # List the spatial bins to analyze
    print(f'Looking at aperture {sp_bin_idx=}')

    print(f'**** Data: {h2d.y.shape = }')
    vx_range = [h2d.x[0].min(), h2d.x[0].max()]
    vy_range = [h2d.x[1].min(), h2d.x[1].max()]
    print(f'{vx_range=}, {vy_range=}')
    print(f"xedg-range (vx): {h2d.xedg[0][0]}, {h2d.xedg[0][-1]}")
    print(f"xedg-range (vy): {h2d.xedg[1][0]}, {h2d.xedg[1][-1]}")
    _ = p.hist2d_plot(h2d, orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)

    print(f'**** Model: {h2d_model.y.shape = }')
    vx_range = [h2d_model.x[0].min(), h2d_model.x[0].max()]
    vy_range = [h2d_model.x[1].min(), h2d_model.x[1].max()]
    print(f'{vx_range=}, {vy_range=}')
    print(f"xedg-range (vx): {h2d_model.xedg[0][0]}, {h2d_model.xedg[0][-1]}")
    print(f"xedg-range (vy): {h2d_model.xedg[1][0]}, {h2d_model.xedg[1][-1]}")
    _ = p.hist2d_plot(h2d_model, orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)

    print(f'**** Model rebinned: {h2d_rebinned.y.shape = }')
    vx_range = [h2d_rebinned.x[0].min(), h2d_rebinned.x[0].max()]
    vy_range = [h2d_rebinned.x[1].min(), h2d_rebinned.x[1].max()]
    print(f'{vx_range=}, {vy_range=}')
    print(f"xedg-range (vx): {h2d_rebinned.xedg[0][0]}, {h2d_rebinned.xedg[0][-1]}")
    print(f"xedg-range (vy): {h2d_rebinned.xedg[1][0]}, {h2d_rebinned.xedg[1][-1]}")
    _ = p.hist2d_plot(h2d_rebinned, orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)


To get an overview, plot the model velocity distributions in each spatial bin...

In [ ]:
# plot pm histograms for all spatial bins
n_bins = h2d_rebinned.y.shape[-1]
ratio = (h2d_rebinned.x[1].max() - h2d_rebinned.x[1].min()) / \
        (h2d_rebinned.x[0].max() - h2d_rebinned.x[0].min())
fig = plt.figure(figsize=(20, 20 * (n_bins // 4 // 4 + 2) * ratio))
for bin_idx in range(0, n_bins, 1):  ##################################
    ax = plt.subplot(n_bins // 4 + (1 if n_bins % 4 > 0 else 0), 4, bin_idx + 1)
    # im = ax.imshow(data, aspect='equal', interpolation='bilinear', #cmap=cm.RdYlGn,
    im = ax.imshow(h2d_rebinned.y[0,:,:,bin_idx],
                   aspect='equal', interpolation='none', cmap='PuBu', #cmap=cm.RdYlGn,
                   origin='lower', extent=vx_range + vy_range,
                   vmax=data.max(), vmin=0)
    fig.colorbar(im, ax=ax, shrink=1)
    ax.set_title(f'Bin {bin_idx}')
plt.show()

In [ ]:
# plot the pm histograms' mean values (doesn't fit at all yet...)
# mostly from plotter.py

def create_plot(dp_args, data):
    # get aperture and bin data
    x = dp_args['x']
    y = dp_args['y']
    dx = dp_args['dx']
    grid = dp_args['idx_bin_to_pix']
    angle_deg = dp_args['angle']
    # Only select the pixels that have a bin associated with them.
    s = np.ravel(np.where((grid >= 0)))

    #fhist, _ = np.histogram(grid[s], bins=len(data))
    #data = data / fhist
    # plot settings
    #data_min = min(data[grid[s]] / max(data))
    #data_max = max(data[grid[s]] / max(data))
    # The galaxy has NOT already rotated with PA to align major axis with x
    map1 = cmr.get_sub_cmap('twilight_shifted', 0.05, 0.6)
    kw_display_pixels1 = dict(pixelsize=dx,
                              angle=angle_deg,
                              colorbar=True,
                              nticks=7,
                              # cmap='sauron')
                              cmap=map1)
    # PLOT THE DATA
    #plt.figure()
    #c_c = data[grid[s]] / max(data)
    c_c = data[grid[s]]
    display_pixels.display_pixels(x, y, c_c,
                                  vmin=min(data[grid[s]]), vmax=max(data[grid[s]]),
                                  label='velocity',
                                  **kw_display_pixels1)
    #plt.gca().set_title(title)
    #plt.show()

fig = plt.figure(figsize=(15, 5))

n_rows, n_cols = 2, 2

mean_vel = h2d.get_mean()  # tuple (mean_velx(n_orbits, n_apertures), mean_vely(n_orbits, n_apertures))

ax = plt.subplot(n_rows, n_cols, 1)
ax.set_title('Data pm x')
mean = mean_vel[0][0]
create_plot(pm.dp_args, mean)

ax = plt.subplot(n_rows, n_cols, 2)
ax.set_title('Data pm y')
mean = mean_vel[1][0]
create_plot(pm.dp_args, mean)

# data = np.einsum('ijkl,i', h2d_orblib.y, orb_weights)[np.newaxis]  # weights-weighted sum
# hist_plot = dyn.kinematics.Histogram2D(xedg=h2d_orblib.xedg, y=data)
mean_vel = h2d_rebinned.get_mean()  # tuple (mean_velx(n_orbits, n_apertures), mean_vely(n_orbits, n_apertures))

ax = plt.subplot(n_rows, n_cols, 3)
ax.set_title('Model pm x')
mean = mean_vel[0][0]
create_plot(pm.dp_args, mean)

ax = plt.subplot(n_rows, n_cols, 4)
ax.set_title('Model pm y')
mean = mean_vel[1][0]
create_plot(pm.dp_args, mean)

In [ ]:
# Plot orblib pm vs input pm for specific sp_bin_idx
sp_bin_idx_list = [25, 48, 70]

for sp_bin_idx in sp_bin_idx_list:
    print(f'{sp_bin_idx = }')
    _ = p.hist2d_plot(h2d_rebinned, orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)
    _ = p.hist2d_plot(h2d, orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)

In [ ]:
import datetime
datetime.datetime.now()

In [ ]:
f=np.load('Francisco/rot_mod_d5kpc_i00deg/kinematics_hist2d.npz')

In [ ]:
f['PSF_weight'], f['PSF_sigma'], f['nstarbin']

In [ ]:
f['PM_2dhist'][80][15,15], f['PM_2dhist_sigma'][80][15,15]

In [ ]:
dict(f).keys()